# Chinook Music Database Analysis

Chinook is a fictional record store where customers can purchase songs. 

In [27]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Data overview

First, we will query our table to get a basic understaning of the data we are working with. 

In [5]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


We can see that our database contains 11 tables. Let's query a few tables to see the type of data that is contained in them.

In [7]:
%%sql
select * 
from customer 
limit 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


The customer table contains a lot of information that will probably be useful to our analysis. Let's take a look at the invoice table too since that will be helpful for our analysis as well. 

In [9]:
%%sql
select *
from invoice
limit 5;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


The total column contains columns information about what each customer has paid. This will also be useful for our analysis.

## Number of Tracks sold in US by Genre

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four: Regal in the Hip-hop genre, Red Tone in the Punk genre, Meteor and the Girls in the Pop genre, and Slim Jim Bites in the Blues genre.

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

First I will isolate american sales.

In [28]:
%%sql

SELECT il.*
 FROM invoice_line AS il
 INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
 INNER JOIN customer AS c ON c.customer_id = i.customer_id
 WHERE c.country = "USA"
limit 10;


Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1


Now I will combine the above subquery with genre information to figure out which genres perform the best.

In [31]:
%%sql

WITH usa_only AS
(SELECT il.*
 FROM invoice_line AS il
 INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
 INNER JOIN customer AS c ON c.customer_id = i.customer_id
 WHERE c.country = "USA")

SELECT  g.name AS genre,
        COUNT(uo.invoice_line_id) AS "num_of_tracks",
        ROUND(CAST(COUNT(uo.invoice_line_id) AS FLOAT) / (SELECT COUNT(*) 
                                                          FROM usa_only) 
              *100, 2) AS "percentage"
FROM usa_only AS uo
INNER JOIN track AS t ON t.track_id = uo.track_id
INNER JOIN genre AS g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC;

Done.


genre,num_of_tracks,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


The most popular genres are rock, punk, and metal. Therefore I would recommend the record company advertise the records in the following order:

* Red Tone (Punk) 
* Slim Jim Bites (Blues)
* Meteor and the Girls (Pop)
* Regal (Hip Hop)

## Sales Support Agent Total Sales

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

First we will isolate the sales support agents in total list of employees. We will include the hire date because that may be important to our analysis.

In [29]:
%%sql
select 
e.first_name || " " || e.last_name as rep_name,
e.employee_id,
e.title,
e.hire_date,
c.support_rep_id
from employee as e
left join customer as c on c.support_rep_id = e.employee_id
where e.title = "Sales Support Agent"
limit 20;


Done.


rep_name,employee_id,title,hire_date,support_rep_id
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3
Jane Peacock,3,Sales Support Agent,2017-04-01 00:00:00,3


Now we will use the above subquerey to get the total amount for all three support reps. 

In [29]:
%%sql

with agent_name as 
    (select
    e.first_name || " " || e.last_name as rep_name,
e.employee_id as employee,
e.title,
e.hire_date,
c.support_rep_id,
c.customer_id
from employee as e
left join customer as c on c.support_rep_id = e.employee_id
where e.title = "Sales Support Agent"
)
    
select
rep_name,
employee,
hire_date,
round(sum(i.total),2) as total
from agent_name as an
left join invoice as i on i.customer_id = an.customer_id
group by 2
order by 3

Done.


rep_name,employee,hire_date,total
Jane Peacock,3,2017-04-01 00:00:00,1731.51
Margaret Park,4,2017-05-03 00:00:00,1584.0
Steve Johnson,5,2017-10-17 00:00:00,1393.92


From our analysis we can see that Jane Peacock is the highest performing sales rep, however she is was hired 6 months before the lowest performing sales rep, Steve Johnson. 

Steve is also only trailing Jane by less than $400, although Jane has has had a 6-month head start. Therefore I reccomend doing this anlysis in another 6 months to get a deeper understanding of the employee performance.

# Sales From Each Customer By Country

Your next task is to analyze the sales data for customers from each different country.

In particular, you have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

First we will explore the data based on customer account to see if there are any trends.

In [48]:
%%sql

SELECT
         country,
         count(customer_id) count
     FROM customer
     GROUP by 1
     ORDER by 2 DESC

Done.


country,count
USA,13
Canada,8
Brazil,5
France,5
Germany,4
United Kingdom,3
Czech Republic,2
India,2
Portugal,2
Argentina,1


Some countries only have one customer. For easier viewing, we want to make sure these counties are all grouped into the "other" category.

In [47]:
%%sql
SELECT
                 CASE 
                    WHEN (SELECT COUNT(*) 
                          FROM customer 
                          WHERE country = c.country) = 1 THEN "Other"
                    ELSE  c.country 
                 END AS "country",
                 c.customer_id,
                 il.*
             FROM 
                 invoice_line AS il
             INNER JOIN 
                 invoice AS i ON i.invoice_id = il.invoice_id
             INNER JOIN 
                 customer AS c ON c.customer_id = i.customer_id
                limit 10;

Done.


country,customer_id,invoice_line_id,invoice_id,track_id,unit_price,quantity
USA,18,1,1,1158,0.99,1
USA,18,2,1,1159,0.99,1
USA,18,3,1,1160,0.99,1
USA,18,4,1,1161,0.99,1
USA,18,5,1,1162,0.99,1
USA,18,6,1,1163,0.99,1
USA,18,7,1,1164,0.99,1
USA,18,8,1,1165,0.99,1
USA,18,9,1,1166,0.99,1
USA,18,10,1,1167,0.99,1


Now we will put all this together and find the requested data.

In [21]:
%%sql
WITH country_or_other AS
            (SELECT
                 CASE 
                    WHEN (SELECT COUNT(*) 
                          FROM customer 
                          WHERE country = c.country) = 1 THEN "Other"
                    ELSE  c.country 
                 END AS "country",
                 c.customer_id,
                 il.*
             FROM 
                 invoice_line AS il
             INNER JOIN 
                 invoice AS i ON i.invoice_id = il.invoice_id
             INNER JOIN 
                 customer AS c ON c.customer_id = i.customer_id)
    
select
country,
cust_count,
total_sales,
avg_cust_spend,
avg_invoice_amt
from
    (select
     country,
    count(distinct customer_id) as cust_count,
    round(sum(unit_price),2) as total_sales,
     round(cast(sum(unit_price) as float) / count(distinct customer_id),2) as 
      avg_cust_spend,
     round(cast(sum(unit_price)as float)/ count(distinct invoice_id),2) as avg_invoice_amt,
     case
         when country = "other" then 1 
    else 0
    end as sort
    from country_or_other
     group by 1
     order by sort asc, 2 desc);


Done.


country,cust_count,total_sales,avg_cust_spend,avg_invoice_amt
Other,15,1094.94,73.0,7.45
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.58,8.72
Portugal,2,185.13,92.57,6.38


Since the other column represents countries with only 1 customer, I will ignore for my analysis. 

I would reccomend the company invests in the following countries based on average customer spend:

* Czech Republic
* Portugal
* India

# Album sales vs. Individual sales

Management's current business model is a dual-approach:

* allow users to purchase entire albums

AND

* allow users to purchase one or more individual tracks

However, they are considering adopting a different purchasing strategy to save money. The new strategy would be purchasing only the most popular tracks from each album from record companies, instead of purchasing entire albums.

You have been asked find out what percentage of purchases are indivual tracks vs whole albums to help inform management's strategy.

First, I will seperate album purchases and individual track purchases.

In [24]:
%%sql
SELECT 
        il.invoice_id,
        CASE
            WHEN COUNT(DISTINCT t.album_id) = 1
                 AND
                 COUNT(DISTINCT t.track_id) =  c.total_num_tracks_in_album THEN 'album'
            ELSE 'track' 
        END AS 'album_or_track'
    FROM 
        track AS t
    INNER JOIN 
        invoice_line AS il ON il.track_id = t.track_id 
    INNER JOIN (SELECT album_id, 
      COUNT(*) as 'total_num_tracks_in_album' 
      FROM track 
     GROUP BY album_id) as c ON c.album_id = t.album_id 
    GROUP BY 1
    limit 10;

Done.


invoice_id,album_or_track
1,album
2,track
3,track
4,track
5,album
6,track
7,track
8,track
9,track
10,track


Now we will combine the above query to find the percentage of albums sold vs. tracks.

In [34]:
%%sql

with album_v_track as
(SELECT 
        il.invoice_id,
        CASE
            WHEN COUNT(DISTINCT t.album_id) = 1
                 AND
                 COUNT(DISTINCT t.track_id) =  c.total_num_tracks_in_album THEN 'album'
            ELSE 'track' 
        END AS 'album_or_track'
    FROM 
        track AS t
    INNER JOIN 
        invoice_line AS il ON il.track_id = t.track_id 
    INNER JOIN (SELECT album_id, 
      COUNT(*) as 'total_num_tracks_in_album' 
      FROM track 
     GROUP BY album_id) as c ON c.album_id = t.album_id 
    GROUP BY 1)

select
album_or_track,
count(*) as num_invoices,
ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM invoice), 2) AS 'percentage'
from album_v_track
group by 1

Done.


album_or_track,num_invoices,percentage
album,114,0.19
track,500,0.81


Album purchases represent 1/5 of total revenue. This means if management was to move forward with this strategy, they would cut their current revenue by almost 20%! 

Therefore, I would not reccomend management moves forward with this strategy. 